# MAG dataset
After downloading the WoS dataset, we complement it with relevant information from the MAG dataset, including received citations, references and author information. Here we show how we process pair-wise matching using the big MAG dataset(s). Note that each MAG dataset has more than a hundred million lines, which we would not have enough memory to read it at one time. 
- We first split each docuement into smaller text files using python.
- We then use Pandas to read each small text file into a DataFrame. We process it in chunks by specifying the chunksize= 100000, and merge the returned TextFileReader objects. 

*For detailed data schema of the MAG dataset(s), see https://docs.microsoft.com/en-us/academic-services/graph/reference-data-schema*

In [ ]:
import pandas as pd
import os

**Find paperid through doi**

In [ ]:
# the Doi list from wos
with open ('list_DOI.txt','r') as f:
    a=f.read()
a=a.lower()
a='\n'+a
a=a.replace('\n','@#')

In [ ]:
len(a.split('@#')[1:-1])

11951

In [ ]:
#search through the mag dataset
n=0
data=pd.DataFrame()
for ii,line in enumerate(open('Papers.txt',encoding='utf8')):
    if ii <100:
        doi=line.lower().strip().split('\t')[2]
        query=str('@#'+doi+'@#')
        if query in a:
            data.loc[n,'DOI']=doi
            data.loc[n,'ID']=line.strip().split('\t')[0]
            data.loc[n,'Title']=line.strip().split('\t')[5]
            n+=1
    else:
        break

In [ ]:
data=data.drop_duplicates(subset=['DOI']).reset_index(drop=True)

In [ ]:
wos=pd.read_csv('wos.csv')
a=pd.merge(wos,data,on=['DOI']).drop_duplicates(subset=['DOI']).dropna(subset=['DOI']).reset_index(drop=True)

In [ ]:
a.to_csv('id_drop1.csv',index=False)

In [ ]:
len(a)/len(wos) #98% articles are found in the MAG dataset

0.9820098736507406

**Find reference paperid**

In [ ]:
#write 10 smaller documents
with open('PaperReferences.txt') as infp:
    files = [open('citation%d.txt' % i, 'w') for i in range(10)]
    for i, line in enumerate(infp):
        files[i % 10].write(line)
    for f in files:
        f.close()

In [ ]:
data=pd.read_csv('id_drop1.csv')

In [ ]:
part=[]
dfs=[]
for i in range(8,10):
    df = pd.read_csv('citation'+str(i)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['ID','reference'])
    chunk_list = []
    for chunk in df:  
        chunk_list.append(chunk)
    df_concat = pd.concat(chunk_list)
    dfs.append(df_concat)
    part.append(pd.merge(data,df_concat,on='ID'))

In [ ]:
# write to csv and save 
part[0].to_csv('citation8.csv',index=False)
part[1].to_csv('citation9.csv',index=False)

In [ ]:
pd.read_csv('citation8.csv')

,DOI,ID,Title,reference
0,10.1016/j.is.2013.09.002,1965015069,Discovering OLAP dimensions in semi-structured...,1500614872
1,10.1016/j.is.2013.09.002,1965015069,Discovering OLAP dimensions in semi-structured...,2101196063
2,10.1016/j.is.2013.09.002,1965015069,Discovering OLAP dimensions in semi-structured...,2151451758
3,10.1017/s1049096518001038,2899452135,Politics @Pontifex: International Crises and P...,2109223572
4,10.1080/10550887.2015.1074505,2160578232,Drug Use in the Twittersphere: A Qualitative C...,2151919021
...,...,...,...,...
38272,10.4018/ijt.2019070104,2947562382,Social Media to Social Media Analytics: Ethica...,2898759334
38273,10.1007/s00521-014-1642-9,2020215385,Study of collective user behaviour in Twitter:...,1976320242
38274,10.1007/s00521-014-1642-9,2020215385,Study of collective user behaviour in Twitter:...,2060907774
38275,10.1007/s00521-014-1642-9,2020215385,Study of collective user behaviour in Twitter:...,2118160723


**Find the year of citing publication**

In [ ]:
import pandas as pd
extend=pd.read_csv('extend.csv')

In [ ]:
df = pd.read_csv('Paper'+str(19)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['citing','Rank','DOI','DocType','PaperTitle','OriginalTitle','BookTitle','Year','Date','OnlineDate','Publisher','JournalId','ConferenceSeriesId','ConferenceInstanceId','Volume','Issue','FirstPage','LastPage','ReferenceCount','CitationCount','EstimatedCitation','OriginalVenue','FamilyId','CreatedDate'])
chunk_list = []
for chunk in df:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)
part=pd.merge(extend,df_concat,on='citing')

In [ ]:
part.to_csv('citing_year19.csv',index=False)

In [ ]:
import pandas as pd
ls=[]
for i in range(20):
    ls.append(pd.read_csv('citing_year'+str(i)+'.csv'))

In [ ]:
citing=pd.concat(ls).reset_index(drop=True)

In [ ]:
citing=citing[['citing','DOI_y','PaperTitle','Year']]

In [ ]:
citing.to_csv('citing_year.csv',index=False)

In [ ]:
extend=citing[~citing['citing'].isin(data.ID)].reset_index(drop=True)

In [ ]:
extend.to_csv('extend.csv',index=False)

**Extend paper category**

In [ ]:
import pandas as pd
data=pd.read_csv('node_extend.csv')
#from paperid find papers 
df = pd.read_csv('Paper'+str(11)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['PaperId','Rank','DOI','DocType','PaperTitle','OriginalTitle','BookTitle','Year','Date','OnlineDate','Publisher','JournalId','ConferenceSeriesId','ConferenceInstanceId','Volume','Issue','FirstPage','LastPage','ReferenceCount','CitationCount','EstimatedCitation','OriginalVenue','FamilyId','CreatedDate'])
chunk_list = []
for chunk in df:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)
df_concat.DOI=df_concat.DOI.str.lower()
part=pd.merge(data,df_concat,on='DOI')

In [ ]:
part.to_csv('coci_extend11.csv',index=False)

In [ ]:
ls=[]
for i in range(20):
    ls.append(pd.read_csv('coci_extend'+str(i)+'.csv'))

In [ ]:
a=pd.concat(ls).reset_index(drop=True)

In [ ]:
28338/len(data) #percentage

0.9530824336595701

In [ ]:
a=a.drop_duplicates(subset=['DOI'])[['DOI', 'PaperId', 'Rank', 'DocType', 'PaperTitle', 'OriginalTitle',
       'BookTitle', 'Year', 'Date',  'Publisher', 'JournalId',
       'ConferenceSeriesId', 'ConferenceInstanceId', 'Volume', 'Issue',
       'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount',
       'EstimatedCitation', 'OriginalVenue', 'FamilyId', 'CreatedDate','OnlineDate']]

In [ ]:
a.columns=['DOI', 'PaperId', 'Rank', 'DocType', 'PaperTitle', 'OriginalTitle',
       'BookTitle', 'Year', 'Date',  'Publisher', 'JournalId',
       'ConferenceSeriesId', 'ConferenceInstanceId', 'Volume', 'Issue',
       'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount',
       'EstimatedCitation', 'OriginalVenue', 'FamilyId', 'CreatedDate','OnlineDate']

In [ ]:
a=a.drop(columns='OnlineDate')

In [ ]:
a.to_csv('coci-references.csv',index=False)

In [ ]:
import pandas as pd
data=pd.read_csv('coci-references.csv')

In [ ]:
#from paperid find fieldofstudyId
articles=[]
dfs=[]
for i in range(6,10):
    df = pd.read_csv('FieldId'+str(i)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['PaperId','FieldId','score'])
    chunk_list = []
    for chunk in df:  
        chunk_list.append(chunk)
    df_concat = pd.concat(chunk_list)
    dfs.append(df_concat)
    articles.append(pd.merge(data,df_concat,on='PaperId'))

In [ ]:
articles[0].to_csv('ref_field6.csv',index=False)
articles[1].to_csv('ref_field7.csv',index=False)
articles[2].to_csv('ref_field8.csv',index=False)
articles[3].to_csv('ref_field9.csv',index=False)

In [ ]:
ls=[]
for i in range(10):
    ls.append(pd.read_csv('ref_field'+str(i)+'.csv'))

In [ ]:
a=pd.concat(ls).reset_index(drop=True)

In [ ]:
a.to_csv('ref_fieldid.csv',index=False)

In [ ]:
import pandas as pd
data=pd.read_csv('ref_fieldid.csv')

In [ ]:
data.head()

,DOI,PaperId,Rank,DocType,PaperTitle,OriginalTitle,BookTitle,Year,Date,Publisher,...,FirstPage,LastPage,ReferenceCount,CitationCount,EstimatedCitation,OriginalVenue,FamilyId,CreatedDate,FieldId,score
0,10.1145/2187836.2187871,2020221730,17784,Conference,dynamical classes of collective attention in t...,Dynamical classes of collective attention in t...,WWW,2012.0,2012-04-16,ACM,...,251.0,260.0,37.0,216.0,285.0,the web conference,2.020222e+09,2016-06-24,86256295,0.485299
1,10.1145/2187836.2187871,2020221730,17784,Conference,dynamical classes of collective attention in t...,Dynamical classes of collective attention in t...,WWW,2012.0,2012-04-16,ACM,...,251.0,260.0,37.0,216.0,285.0,the web conference,2.020222e+09,2016-06-24,143275388,0.549903
2,10.1145/2187836.2187871,2020221730,17784,Conference,dynamical classes of collective attention in t...,Dynamical classes of collective attention in t...,WWW,2012.0,2012-04-16,ACM,...,251.0,260.0,37.0,216.0,285.0,the web conference,2.020222e+09,2016-06-24,518677369,0.000000
3,10.1037/0033-2909.108.3.480,2149893809,17502,Journal,the case for motivated reasoning,The case for motivated reasoning.,NaN,1990.0,1990-01-01,American Psychological Association,...,480.0,498.0,100.0,4086.0,6692.0,Psychological Bulletin,NaN,2016-06-24,120936955,0.478684
4,10.1037/0033-2909.108.3.480,2149893809,17502,Journal,the case for motivated reasoning,The case for motivated reasoning.,NaN,1990.0,1990-01-01,American Psychological Association,...,480.0,498.0,100.0,4086.0,6692.0,Psychological Bulletin,NaN,2016-06-24,143299363,0.510120


In [ ]:
#from fieldofstudyId to fieldofstudy
df = pd.read_csv('FieldsofStudy.txt', sep='\t', iterator=True, chunksize=1000000,names=['FieldId','Rank','NormalizedName','DisplayName','MainType','Level','PaperCount','PaperFamilyCount','CitationCount','CreatedDate'])
chunk_list = []
for chunk in df:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)
field=pd.merge(data,df_concat,on='FieldId')

In [ ]:
field_1=field[['DOI','PaperTitle','Year','Publisher','Level','score','NormalizedName']]

In [ ]:
field_1 = field_1.astype(str)

In [ ]:
first=field_1.groupby('DOI').agg({'PaperTitle':'first',
                                 'Year': 'first', 
                         'Publisher': 'first',
                                  'score': '; '.join,
                                  'Level': '; '.join,
                            'NormalizedName':'; '.join}).reset_index()

In [ ]:
first.to_csv('referece_disc.csv',index=False)

**Author**

In [ ]:
#articles that found in mag
data=pd.read_csv('id_drop1.csv')

In [ ]:
data.columns=['DOI', 'PaperId', 'Title']

In [ ]:
#find all publications from the authors in mag
ls=[]
for i in range(10):
    ls.append(pd.read_csv('part'+str(i)+'.csv'))

In [ ]:
a=pd.concat(ls).reset_index(drop=True)

In [ ]:
data = data.astype(str)

In [ ]:
#group by author id
author_pub=data.groupby('AuthorId').agg({'OriginalAuthor_x': 'first',
                              'OriginalAuthor_y': 'first',
                             'PaperId': '; '.join, 
                                'AffiliationId': '; '.join,
                                 'OriginalAffiliation': '; '.join,
                                'AuthorSequenceNumber': '; '.join}).reset_index()

In [ ]:
#calculate the number of publications
for i in range(len(author_pub)):
    author_pub.loc[i,'Number_of_pub']=str(len(author_pub.PaperId[i].split('; ')))

In [ ]:
#rename columns
author_pub.columns=['AuthorId', 'OriginalAuthor_x', 'OriginalAuthor_y', 'Full_PaperId',
       'Full_AffiliationId', 'Full_OriginalAffiliation', 'Full_AuthorSequenceNumber',
       'Number_of_pubs']

In [ ]:
author_pub.to_csv('author_publications.csv',index=False)

In [ ]:
#author basic information
data_1=pd.read_csv('author_information.csv')

In [ ]:
data.columns=['AuthorId', 'OriginalAuthor', 'DOI', 'PaperId', 'Title',
       'AffiliationId', 'OriginalAffiliation', 'AuthorSequenceNumber', 'Rank',
       'NormalizedName', 'DisplayName', 'LastKnownAffiliationId', 'PaperCount',
        'CitationCount', 'CreatedDate','n']

In [ ]:
data_1=data_1.astype('str')

In [ ]:
a=pd.merge(author_pub,data_1,on='AuthorId',how='outer')

In [ ]:
a.to_csv('author_publications_merged.csv',index=False)

In [ ]:
author = author.astype(str)

In [ ]:
author_norm=author.groupby('AuthorId').agg({'OriginalAuthor': 'first',
                            'DOI':'; '.join, 
                             'PaperId': '; '.join, 
                             'Title': '; '.join,
                                'AffiliationId': '; '.join,
                                 'OriginalAffiliation': '; '.join,
                                'AuthorSequenceNumber': '; '.join}).reset_index()

In [ ]:
#author found in paperauthor document
author_norm.to_csv('author_mag.csv',index=False)

In [ ]:
#write 10 smaller documents
with open('PaperAuthorAffiliations.txt',encoding='utf8') as infp:
    files = [open('Aff%d.txt' % i, "w", encoding="utf-8") for i in range(10)]
    for i, line in enumerate(infp):
        files[i % 10].write(line)
    for f in files:
        f.close()

In [ ]:
with open('Authors.txt',encoding='utf8') as infp:
    files = [open('Author%d.txt' % i, "w", encoding="utf-8") for i in range(10)]
    for i, line in enumerate(infp):
        files[i % 10].write(line)
    for f in files:
        f.close()

In [ ]:
data_2=data_1[['AuthorId','OriginalAuthor']]

In [ ]:
df = pd.read_csv('Aff'+str(9)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['PaperId','AuthorId','AffiliationId','AuthorSequenceNumber','OriginalAuthor','OriginalAffiliation'])
chunk_list = []
for chunk in df:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

In [ ]:
part=pd.merge(data_2,df_concat,on='AuthorId')

In [ ]:
part.to_csv('author_pub9.csv',index=False)

In [ ]:
#authors that found in paperauthor document but not in author document
notfound=data_1[~data_1.AuthorId.isin(data.AuthorId)]

In [ ]:
#chunk match
dfs=[]
parts=[]
for i in range(3):
    df = pd.read_csv('Author'+str(i)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['AuthorId','Rank','NormalizedName','DisplayName','LastKnownAffiliationId','PaperCount','CitationCount','CreatedDate'])
    chunk_list = []
    for chunk in df:  
        chunk_list.append(chunk)
    df_concat = pd.concat(chunk_list)
    dfs.append(df_concat)
    parts.append(pd.merge(notfound,df_concat,on='AuthorId'))

Empty DataFrame
Columns: [AuthorId, OriginalAuthor, DOI, PaperId, Title, AffiliationId, OriginalAffiliation, AuthorSequenceNumber, Rank, NormalizedName, DisplayName, LastKnownAffiliationId, PaperCount, PaperFamilyCount, CitationCount, CreatedDate]
Index: []


In [ ]:
len(notfound)

38

**Test author name disambiguation**

In [ ]:
import pandas as pd
data=pd.read_csv('author_mag.csv')

In [ ]:
a=data.drop_duplicates(subset=['OriginalAuthor'])

In [ ]:
b=data[~data.AuthorId.isin(a.AuthorId)].reset_index(drop=True)

In [ ]:
import random
ls=[random.randint(0,974) for i in range(10)]

In [ ]:
ls

[948, 803, 319, 695, 473, 35, 361, 477, 103, 535]

In [ ]:
l=[b.OriginalAuthor[i] for i in ls]

In [ ]:
l_=[]
for i,j in data[data.OriginalAuthor.isin(l)].groupby('OriginalAuthor'):
    l_.append(j)
disa=pd.concat(l_)

In [ ]:
#merge the authors with the same original name
publi=pub.groupby('OriginalAuthor_x').agg({'AuthorId': '; '.join,
                                       'OriginalAuthor_y': '; '.join,
                            'Doi':'; '.join, 
                                        'Year': '; '.join,
                                       'Date': '; '.join,
                            'Rank':'; '.join,
                            'DocType':'; '.join,
                             'PaperId': '; '.join,
                              'PaperTitle': '; '.join,'OriginalTitle': '; '.join,'BookTitle': '; '.join,
                                       'Publisher': '; '.join,
                                       'JournalId': '; '.join,
                                       'ConferenceSeriesId': '; '.join,
                            'ConferenceInstanceId': '; '.join,  
                             'Volume': '; '.join,
                                'Issue': '; '.join,
                                'FirstPage': '; '.join,
                                'LastPage': '; '.join,
                                'ReferenceCount': '; '.join,
                                       'CitationCount': '; '.join,
                                       'EstimatedCitation': '; '.join,
                                       'OriginalVenue': '; '.join,
                                       'CreatedDate': '; '.join,
                                       'FamilyId': '; '.join,
                                'AffiliationId': '; '.join,
                                 'OriginalAffiliation': '; '.join,
                                'AuthorSequenceNumber': '; '.join}).reset_index()

**Articles**

In [ ]:
#write 20 smaller documents
with open('Papers.txt',encoding='utf8') as infp:
    files = [open('Paper%d.txt' % i, 'w',encoding='utf8') for i in range(20)]
    for i, line in enumerate(infp):
        files[i % 20].write(line)
    for f in files:
        f.close()

In [ ]:
#get the full paper ids
data=pd.read_csv('author_publications.csv')
data['Full_PaperId']=data.Full_PaperId.str.split('; ')
papers=data.Full_PaperId.explode().dropna()
papers.to_csv('full_id.csv',index=False)

In [ ]:
import pandas as pd
data=pd.read_csv('full_id.csv')

In [ ]:
#from paperid find papers
df = pd.read_csv('Paper'+str(0)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['Full_PaperId','Rank','Doi','DocType','PaperTitle','OriginalTitle','BookTitle','Year','Date','OnlineDate','Publisher','JournalId','ConferenceSeriesId','ConferenceInstanceId','Volume','Issue','FirstPage','LastPage','ReferenceCount','CitationCount','EstimatedCitation','OriginalVenue','FamilyId','CreatedDate'])
chunk_list = []
for chunk in df:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)
part=pd.merge(data,df_concat,on='Full_PaperId')

In [ ]:
part.to_csv('author_background_publications19.csv',index=False)

In [ ]:
#find academic age from the authors in mag
ls=[]
for i in range(20):
    ls.append(pd.read_csv('author_background_publications'+str(i)+'.csv'))
articles=pd.concat(ls).reset_index(drop=True)

In [ ]:
len(articles.Full_PaperId.unique())

1278331

In [ ]:
#find all publications from the authors in mag
ls=[]
for i in range(10):
    ls.append(pd.read_csv('author_pub'+str(i)+'.csv'))
author=pd.concat(ls).reset_index(drop=True)

In [ ]:
articles.columns=['PaperId', 'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle',
       'BookTitle', 'Year', 'Date', 'OnlineDate', 'Publisher', 'JournalId',
       'ConferenceSeriesId', 'ConferenceInstanceId', 'Volume', 'Issue',
       'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount',
       'EstimatedCitation', 'OriginalVenue', 'FamilyId', 'CreatedDate']

In [ ]:
author=author.astype(str)
articles=articles.astype(str)

In [ ]:
merge=author.merge(articles,on='PaperId',how='left')

In [ ]:
merge.columns=['AuthorId', 'OriginalAuthor_x', 'PaperId', 'AffiliationId',
       'AuthorSequenceNumber', 'OriginalAuthor_y', 'OriginalAffiliation',
       'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle', 'BookTitle',
       'Year', 'Date', 'Publisher', 'JournalId',
       'ConferenceSeriesId', 'ConferenceInstanceId', 'Volume', 'Issue',
       'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount',
       'EstimatedCitation', 'OriginalVenue', 'FamilyId', 'CreatedDate','OnlineDate']

In [ ]:
merge=merge.drop(columns=['OnlineDate'])

In [ ]:
merge.columns

Index(['AuthorId', 'OriginalAuthor_x', 'PaperId', 'AffiliationId',
       'AuthorSequenceNumber', 'OriginalAuthor_y', 'OriginalAffiliation',
       'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle', 'BookTitle',
       'Year', 'Date', 'Publisher', 'JournalId', 'ConferenceSeriesId',
       'ConferenceInstanceId', 'Volume', 'Issue', 'FirstPage', 'LastPage',
       'ReferenceCount', 'CitationCount', 'EstimatedCitation', 'OriginalVenue',
       'FamilyId', 'CreatedDate'],
      dtype='object')

In [ ]:
author = author.astype(str)

In [ ]:
merge=merge.astype(str)
merge=merge.drop_duplicates()

In [ ]:
merge_1=merge.groupby('AuthorId').agg({
                                       'OriginalAuthor_x': 'first',
                            'Doi':'; '.join, 
                                        'Year': '; '.join,
                                       'Date': '; '.join,
                             'PaperId': '; '.join,
                              'PaperTitle': '; '.join,'OriginalTitle': '; '.join,
                                       'Publisher': '; '.join,
                                       'JournalId': '; '.join,
                                       'ConferenceSeriesId': '; '.join,
                                'ReferenceCount': '; '.join,
                                       'CitationCount': '; '.join,
                                       'EstimatedCitation': '; '.join,
                                       'OriginalVenue': '; '.join,
                                       'CreatedDate': '; '.join,
                                'AffiliationId': '; '.join,
                                 'OriginalAffiliation': '; '.join,
                                'AuthorSequenceNumber': '; '.join}).reset_index()

In [ ]:
merge_1.shape

(29458, 19)

In [ ]:
merge_1['AcademicAge']=merge_1.Year

In [ ]:
merge_1=merge_1.astype(str)

In [ ]:
for i in range(len(merge_1)):
    try:
        merge_1.loc[i,'AcademicAge']=min([int(float(j[:4])) for j in merge_1.AcademicAge.to_list()[i].split('; ') if j not in 'nan'])
    except:
        pass

In [ ]:
merge_1.AcademicAge.unique()

array([1996, 1998, 1997, 2010, 2009, 2007, 2012, 2011, 1989, 2015, 2006,
       2003, 1991, 2001, 2004, 1994, 2008, 2005, 2002, 2013, 1999, 1977,
       1970, 1995, 2000, 1993, 1990, 1988, 1971, 1984, 1983, 1982, 2014,
       1965, 1985, 1987, 1992, 1978, 1986, 1868, 1980, 1800, 1975, 2017,
       1981, 2016, 1979, 1974, 1973, 1976, 1969, 2019, 1879, 1972, 2018,
       1968, 1967, 1964, 1900, 'nan', 1966, 1962, 1959, 1847, 1956, 1944,
       1929, 1903, 1927, 1899, 1896, 1943, 1854, 1932, 1945, 1928, 1957,
       1930, 1891, 1862, 1953, 1958, 1963, 1911, 1939, 1961, 1942, 1870,
       1831, 1920, 1941, 1926, 1924, 1851, 1960, 1824, 1938, 1937, 1936,
       1828, 1846, 'nan; nan', 1855, 2020, 1848, 1935, 1949], dtype=object)

In [ ]:
merge_1.columns=['AuthorId', 'OriginalAuthor_x', 'OriginalAuthor_y', 'Doi', 'Year',
       'Date', 'Rank', 'DocType', 'PaperId', 'PaperTitle', 'OriginalTitle',
       'BookTitle', 'Publisher', 'JournalId', 'ConferenceSeriesId',
       'ConferenceInstanceId', 'Volume', 'Issue', 'FirstPage', 'LastPage',
       'ReferenceCount', 'CitationCount', 'EstimatedCitation', 'OriginalVenue',
       'CreatedDate', 'FamilyId', 'AffiliationId', 'OriginalAffiliation',
       'AuthorSequenceNumber', 'AcademicAgeSince']

In [ ]:
merge_1.to_csv('author_academic_age.csv',index=False)

In [ ]:
merge_1.dropna(subset=['AcademicAgeSince'])

In [ ]:
#test if the articles counts are right
import pandas as pd
data=pd.read_csv('author_information.csv')

In [ ]:
data_1[data_1.OriginalAuthor=='Kingsley Purdam']

,AuthorId,OriginalAuthor,DOI,PaperId,Title,AffiliationId,OriginalAffiliation,AuthorSequenceNumber,Rank,NormalizedName,DisplayName,LastKnownAffiliationId,PaperCount,CitationCount,CreatedDate
26466,100249880,Kingsley Purdam,10.1177/0011392114527997,2078729300,Citizen social science and citizen data? Metho...,28407311.0,"University of Manchester, UK.#TAB#",1,16010,kingsley purdam,Kingsley Purdam,28407311.0,99,494,2016-06-24


**First article discipline**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pub=pd.read_csv('author_academic_age.csv')

In [ ]:
pub.to_csv('author_academic_age.csv', index=False)

In [ ]:
pub['AcademicAge']=pub.Year

In [ ]:
pub = pub.astype(str)

In [ ]:
for i in range(len(articles.Year)):
    n=articles.Year[i]
    articles.loc[i,'Year']=str(n).replace(n,n[:4])       

In [ ]:
for n in range(len(pub)):
    try:
        a=np.argmin([int(float(i[:4])) for i in pub.Year.to_list()[n].replace('nan', '9999').split('; ') if i not in 'nan'])
        pub.loc[n,'FirstId']=pub.PaperId[n].split('; ')[a]
        pub.loc[n,'FirstDOI']=pub.Doi[n].split('; ')[a]
        pub.loc[n,'FirstTitle']=pub.PaperTitle[n].split('; ')[a]
        pub.loc[n,'FirstYear']=pub.Year[n].split('; ')[a]
    except:
        pub.loc[n,'FirstId']=pub.PaperId[n]
        pub.loc[n,'FirstDOI']=pub.Doi[n]
        pub.loc[n,'FirstTitle']=pub.PaperTitle[n]
        pub.loc[n,'FirstYear']=pub.Year[n]

In [ ]:
#write 10 smaller documents
with open('PaperFieldsofStudy.txt',encoding='utf8') as infp:
    files = [open('FieldId%d.txt' % i, "w", encoding="utf-8") for i in range(10)]
    for i, line in enumerate(infp):
        files[i % 10].write(line)
    for f in files:
        f.close()

In [ ]:
first_1.to_csv('FirstId_1.csv',index=False)

In [ ]:
import pandas as pd
first=pd.read_csv('FirstId_1.csv')
data=pd.read_csv('Id_drop2.csv')

In [ ]:
#from paperid find fieldofstudyId
firsts=[]
dfs=[]
for i in range(4,7):
    df = pd.read_csv('FieldId'+str(i)+'.txt', sep='\t', iterator=True, chunksize=1000000,names=['FirstId','FieldId','score'])
    chunk_list = []
    for chunk in df:  
        chunk_list.append(chunk)
    df_concat = pd.concat(chunk_list)
    dfs.append(df_concat)
    firsts.append(pd.merge(first,df_concat,on='FirstId'))

In [ ]:
firsts[0].to_csv('first_filed4.csv',index=False)
firsts[1].to_csv('first_filed5.csv',index=False)
firsts[2].to_csv('first_filed6.csv',index=False)

In [ ]:
#find academic age from the authors in mag
ls=[]
for i in range(10):
    ls.append(pd.read_csv('first_filed'+str(i)+'.csv'))
firsts=pd.concat(ls).reset_index(drop=True)

In [ ]:
#from fieldofstudyId to fieldofstudy
df = pd.read_csv('FieldsofStudy.txt', sep='\t', iterator=True, chunksize=1000000,names=['FieldId','Rank','NormalizedName','DisplayName','MainType','Level','PaperCount','PaperFamilyCount','CitationCount','CreatedDate'])
chunk_list = []
for chunk in df:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)
first=pd.merge(firsts,df_concat,on='FieldId')

In [ ]:
first=first.drop_duplicates().reset_index(drop=True)

In [ ]:
first=first.drop(columns=['CreatedDate'])

In [ ]:
first.columns=['ID', 'FieldId', 'score', 'Rank', 'NormalizedName', 'DisplayName',
       'MainType', 'Level', 'PaperCount', 'CitationCount','CreatedDate']

In [ ]:
first=first.astype(str)

In [ ]:
first_1=first.groupby('ID').agg({
                                 'FieldId': '; '.join, 
                         'score': '; '.join,
                          'Rank': '; '.join,
                            'NormalizedName':'; '.join, 
                                        'DisplayName': '; '.join,
                                       'Level': '; '.join,
                               'PaperCount': '; '.join,
                               'CitationCount': '; '.join,
                              'CreatedDate': '; '.join}).reset_index()

In [ ]:
first_1.to_csv('first_article_category.csv',index=False)

In [ ]:
first=pd.read_csv('first_article_category.csv')

In [ ]:
first=first[['ID','Level','score','NormalizedName']]

In [ ]:
first.columns=['FirstId', 'Level','score', 'CategoryName']

In [ ]:
pub_1=pub[['OriginalAuthor_x', 'AuthorId','AcademicAge', 'FirstId']]

In [ ]:
pub_1=pub_1.astype(str)
first_1=first_1.astype(str)

In [ ]:
first_2=pub_1.merge(first_1,on=['FirstId'],how='outer')

In [ ]:
data=pd.read_csv('author_mag.csv')

In [ ]:
data_1=pd.read_csv('author_information.csv')

In [ ]:
data_2=data.merge(data_1,on=['AuthorId'],how='outer')

In [ ]:
author=data_2[['AuthorId', 'OriginalAuthor_x', 'DOI_x', 'PaperId_x', 'Title_x',
       'AffiliationId_x', 'OriginalAffiliation_x', 'AuthorSequenceNumber_x','Rank',
       'NormalizedName', 'DisplayName', 'LastKnownAffiliationId', 'PaperCount',
       'CitationCount', 'CreatedDate']]

In [ ]:
author.columns=['AuthorId', 'OriginalAuthor_x', 'DOI', 'PaperId', 'Title',
       'AffiliationId', 'OriginalAffiliation', 'AuthorSequenceNumber',
       'Rank', 'NormalizedName', 'DisplayName', 'LastKnownAffiliationId',
       'PaperCount', 'CitationCount', 'CreatedDate']
author=author.astype(str)

In [ ]:
author_1=author.groupby('AuthorId').agg({
    'OriginalAuthor_x':'first',
                            'DOI':'; '.join, 
                             'PaperId': '; '.join,
                              'Title': '; '.join,'AffiliationId': '; '.join,'OriginalAffiliation': '; '.join,
                                       'AuthorSequenceNumber': '; '.join,
                                       'Rank': '; '.join,
                                       'NormalizedName': 'first',
                            'LastKnownAffiliationId': '; '.join,
                                            'PaperCount': '; '.join,
                                       'CitationCount': '; '.join,'CreatedDate':': '.join}).reset_index()

In [ ]:
author_1=author_1[['AuthorId','OriginalAuthor_x', 'DOI', 'PaperId', 'Title',
       'AffiliationId', 'OriginalAffiliation', 'AuthorSequenceNumber', 'Rank',
       'NormalizedName', 'LastKnownAffiliationId', 'PaperCount',
       'CitationCount', 'CreatedDate']]

In [ ]:
first_2=first_2[['AuthorId', 'AcademicAge', 'FirstId',
       'Level','score', 'CategoryName']]

In [ ]:
author_3=author_1.merge(first_2,on=['AuthorId'])

In [ ]:
author_3.to_csv('author_full.csv',index=False)

In [ ]:
author_3=pd.read_csv('author_full.csv')

In [ ]:
first_2=first_2[['OriginalAuthor_x','Level']]

In [ ]:
author_3=author_3.astype(str)
first_2=first_2.astype(str)

In [ ]:
author_3=author_3.merge(first_2,on=['OriginalAuthor_x'])

In [ ]:
author=pd.read_csv('author_full_1.csv')

In [ ]:
merge.to_csv('author_aca_age_disc.csv',index=False)

In [ ]:
a=pub[pub['AcademicAgeSince'].isna()] #incomplete data in the author and article txt

In [ ]:
a.head()

,AuthorId,OriginalAuthor_x,OriginalAuthor_y,Doi,Year,Date,Rank,DocType,PaperId,PaperTitle,...,ReferenceCount,CitationCount,EstimatedCitation,OriginalVenue,CreatedDate,FamilyId,AffiliationId,OriginalAffiliation,AuthorSequenceNumber,AcademicAgeSince
3266,2034664642,Adİl Sarıbay,Adİl Sarıbay,NaN,NaN,NaN,NaN,NaN,2150887989,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4405392.0,Bogaziçi University (Turkey),3,NaN
5588,2106695051,Ala Alnaser,Ala Alnaser,NaN,NaN,NaN,NaN,NaN,1979495231,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,189590672.0,"Department of Mathematics, Kansas State Univer...",1,NaN
9295,2151355060,Meltem Kurutaş,Meltem Kurutaş,NaN,NaN,NaN,NaN,NaN,2150887989,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4405392.0,Bogaziçi University (Turkey),5,NaN
9508,2154208150,Tanuka Mukherjee,Tanuka Mukherjee,NaN,NaN,NaN,NaN,NaN,2098925955,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1327163397.0,State University of New York,4,NaN
12732,2272435586,Rena P. Dixon,Rena P. Dixon,NaN,NaN,NaN,NaN,NaN,2590390843,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Carolina Campaign to Prevent Teen Pregna...,3,NaN


In [ ]:
data=pd.read_csv('author_mag.csv')

In [ ]:
data[data.AuthorId==2034664642]

,AuthorId,OriginalAuthor,DOI,PaperId,Title,AffiliationId,OriginalAffiliation,AuthorSequenceNumber
3266,2034664642,Adİl Sarıbay,10.1002/ejsp.2041,2150887989,Would you post that picture and let your dad s...,4405392.0,Bogaziçi University (Turkey),3


In [ ]:
a.iloc[0]

AuthorId                                  2034664642
OriginalAuthor_x                        Adİl Sarıbay
OriginalAuthor_y                        Adİl Sarıbay
Doi                                              NaN
Year                                             NaN
Date                                             NaN
Rank                                             NaN
DocType                                          NaN
PaperId                                   2150887989
PaperTitle                                       NaN
OriginalTitle                                    NaN
BookTitle                                        NaN
Publisher                                        NaN
JournalId                                        NaN
ConferenceSeriesId                               NaN
ConferenceInstanceId                             NaN
Volume                                           NaN
Issue                                            NaN
FirstPage                                     